# 2.2 Prompt Templates in LangChain

## 🎯 Learning Objectives

In this notebook, you'll learn how to create **dynamic, reusable prompts** using LangChain's template system:

1. **PromptTemplate Basics** - Create parameterized prompts with variables
2. **Few-Shot Prompting** - Improve LLM outputs with examples
3. **Chain-of-Thought Prompting** - Guide reasoning step-by-step
4. **Prompt Composition** - Build complex prompts from reusable parts
5. **Serialization** - Save and load prompts from files

## 💡 Why Use Prompt Templates?

Instead of hardcoding prompts like:
```python
SystemMessage(content="You are a helpful assistant that translates English to Spanish.")
```

You can make them dynamic:
```python
template = "You are a helpful assistant that translates {input_language} to {output_language}"
```

---

In [ ]:
# ============================================================================
# ENVIRONMENT SETUP: Load API Keys & Import Dependencies
# ============================================================================
# We use python-dotenv to securely load API keys from a .env file
# This is a best practice - never hardcode API keys in your notebooks!
# ============================================================================

from dotenv import load_dotenv
import os
import sys
import platform

# Load environment variables from .env file
load_dotenv()

# Add parent directory to path for importing helpers
sys.path.append(os.path.abspath("../.."))

# Import our LLM factory functions
# - get_groq_llm(): Creates a Groq-hosted LLM (fast inference with open-source models)
# - get_openai_llm(): Creates an OpenAI GPT model
# - get_databricks_llm(): Creates a Databricks-hosted LLM
from helpers.utils import get_groq_llm, get_openai_llm, get_databricks_llm

print("✅ Environment variables loaded successfully!")
print(f"📍 Running on: {platform.system()}")

# -----------------------------------------------------------------------------
# Initialize the LLM based on platform or preference
# The choice of LLM affects tool calling capabilities and speed
# -----------------------------------------------------------------------------
if sys.platform == "win32":
    # Windows: Use Groq for fast inference
    llm = get_groq_llm()
elif sys.platform == "darwin":
    # macOS: Use Databricks-hosted Gemini
    llm = get_databricks_llm("databricks-gpt-5-1")  
else:
    # Linux: Default to Groq
    llm = get_groq_llm()

# Print which LLM we're using
if hasattr(llm, 'model_name'):
    print(f"🤖 LLM initialized: {llm.model_name}")
elif hasattr(llm, 'model'):
    print(f"🤖 LLM initialized: {llm.model}")
else:
    print("🤖 LLM initialized successfully")

In [ ]:
# ============================================================================
# BASIC PROMPT TEMPLATE
# ============================================================================
# A PromptTemplate uses Python's str.format() syntax with {variable_name}
# Variables are replaced with actual values when you call .format()
# ============================================================================

TEMPLATE = """
You are a helpful assistant that translates the {input_language} to {output_language}
"""

print("Template with placeholders:")
print(TEMPLATE)

In [ ]:
# ============================================================================
# CREATING A PROMPT TEMPLATE
# ============================================================================
# Method 1: .from_template() - Auto-detects variables from the template string
# This is the quickest way when your template is straightforward
# ============================================================================

from langchain_core.prompts import ChatPromptTemplate,PromptTemplate

# Create template - variables are automatically extracted from {placeholders}
prompt_template = PromptTemplate.from_template(template=TEMPLATE)

# Format the template with actual values
formatted_prompt = prompt_template.format(input_language="english", output_language="german")

print("📝 Formatted Prompt:")
print(formatted_prompt)

### Method 2: Explicit Variable Declaration

Passing `input_variables` to the constructor provides **validation** - LangChain will raise an error if you forget a variable or have a typo.

In [ ]:
# ============================================================================
# METHOD 2: Explicit Variable Declaration (Recommended for Production)
# ============================================================================
# By specifying input_variables, you get:
# - Validation that all variables exist in the template
# - Clear documentation of required inputs
# - Early error detection for typos
# ============================================================================

prompt_template = PromptTemplate(
    template=TEMPLATE, 
    input_variables=["input_language", "output_language"]
)

# This will work
print(prompt_template.format(input_language="english", output_language="german"))

# Uncommenting below would raise an error (missing variable):
# prompt_template.format(input_language="english")  # KeyError!


---

## 📚 Few-Shot Prompting

**Few-shot prompting** means providing examples in your prompt to guide the LLM's behavior. This technique:
- Improves output consistency and quality
- Teaches the model your expected format
- Reduces ambiguity in complex tasks

### Use Case: Sentiment Analysis with Subject Extraction

In [ ]:
# ============================================================================
# ZERO-SHOT PROMPT (No Examples)
# ============================================================================
# This template asks the LLM to analyze sentiment WITHOUT providing examples
# It works, but the output format may be inconsistent
# ============================================================================

TEMPLATE_ZERO_SHOT = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Format the output as JSON with the following keys:
sentiment
subject

text: {input}
"""

print("📋 Zero-shot template (no examples):")
print(TEMPLATE_ZERO_SHOT)

### Adding Examples to Improve Quality

To improve performance and consistency, we provide **examples** that show the model exactly what output format we expect. This is called **few-shot prompting**.

In [ ]:
# ============================================================================
# FEW-SHOT PROMPT (With Examples)
# ============================================================================
# This template includes 9 examples covering:
# - 3 different restaurants (BellaVista, SeoulSavor, MunichMeals)
# - 3 sentiment types each (positive, neutral, negative)
# 
# Benefits of few-shot:
# - Consistent output format
# - Model learns your classification criteria
# - Reduces hallucinations
# ============================================================================

TEMPLATE_FEW_SHOT = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Format the output as JSON with the following keys:
sentiment
subject

text: {input}

Examples:
text: The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable.
sentiment: positive
subject: BellaVista

text: BellaVista restaurant was alright. The food was decent, but nothing stood out.
sentiment: neutral
subject: BellaVista

text: I was disappointed with BellaVista. The service was slow and the dishes lacked flavor.
sentiment: negative
subject: BellaVista

text: SeoulSavor offered the most authentic Korean flavors I've tasted outside of Seoul. The kimchi was perfectly fermented and spicy.
sentiment: positive
subject: SeoulSavor

text: SeoulSavor was okay. The bibimbap was good but the bulgogi was a bit too sweet for my taste.
sentiment: neutral
subject: SeoulSavor

text: I didn't enjoy my meal at SeoulSavor. The tteokbokki was too mushy and the service was not attentive.
sentiment: negative
subject: SeoulSavor

text: MunichMeals has the best bratwurst and sauerkraut I've tasted outside of Bavaria. Their beer garden ambiance is truly authentic.
sentiment: positive
subject: MunichMeals

text: MunichMeals was alright. The weisswurst was okay, but I've had better elsewhere.
sentiment: neutral
subject: MunichMeals

text: I was let down by MunichMeals. The potato salad lacked flavor and the staff seemed uninterested.
sentiment: negative
subject: MunichMeals
"""

print(f"✅ Few-shot template created with examples for 3 restaurants x 3 sentiments = 9 examples")


In [ ]:
# ============================================================================
# USING THE FEW-SHOT TEMPLATE
# ============================================================================

prompt_template = PromptTemplate(template=TEMPLATE_FEW_SHOT, input_variables=["input"])

# Format with a new review to analyze
formatted_prompt = prompt_template.format(input="The MunichDeals experience was just awesome!")

print("📝 Formatted prompt (truncated for display):")
print(formatted_prompt[1:300])  # Show last 200 chars to see the input

### Using FewShotPromptTemplate (Modular Approach)

LangChain provides `FewShotPromptTemplate` for a more **structured and maintainable** way to manage examples:
- Examples are stored as a list of dictionaries
- Easy to add, remove, or modify examples
- Cleaner separation of concerns

In [ ]:
# ============================================================================
# FEWSHOTPROMPTTEMPLATE - Modular Example Management
# ============================================================================
# Instead of embedding examples in a string, store them as structured data
# This makes it easy to:
# - Add/remove examples programmatically
# - Load examples from a database or file
# - Dynamically select relevant examples
# ============================================================================

from langchain_core.prompts import ChatPromptTemplate,PromptTemplate,FewShotPromptTemplate

# Examples stored as a list of dictionaries
examples = [
    {
        "text": "The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable.",
        "response": "sentiment: positive\nsubject: BellaVista"
    },
    {
        "text": "BellaVista restaurant was alright. The food was decent, but nothing stood out.",
        "response": "sentiment: neutral\nsubject: BellaVista"
    },
    # Note: Additional examples can be added here...
]

print(f"✅ Loaded {len(examples)} examples")

In [ ]:
# ============================================================================
# DYNAMICALLY ADDING EXAMPLES
# ============================================================================
# You can easily add new examples at runtime
# ============================================================================

new_example = {
    "text": "SeoulSavor was okay. The bibimbap was good but the bulgogi was a bit too sweet for my taste.",
    "response": "sentiment: neutral\nsubject: SeoulSavor"
}
examples.append(new_example)

print(f"✅ Added new example. Total examples: {len(examples)}")

In [ ]:
# ============================================================================
# EXAMPLE TEMPLATE
# ============================================================================
# This template defines how each example will be formatted
# The FewShotPromptTemplate will apply this to each example in the list
# ============================================================================

example_prompt = PromptTemplate(
    input_variables=["text", "response"], 
    template="Text: {text}\n{response}"
)

# Preview how one example looks when formatted
print("📋 Example format preview:")
print(example_prompt.format(**examples[0]))

In [ ]:
# ============================================================================
# BUILDING THE FEWSHOTPROMPTTEMPLATE
# ============================================================================
# Components:
# - examples: List of example dictionaries
# - example_prompt: Template for formatting each example
# - suffix: Text that comes after all examples (contains the actual input)
# - input_variables: Variables in the suffix that need values
# ============================================================================

prompt = FewShotPromptTemplate(
    examples=examples,           # Our list of examples
    example_prompt=example_prompt,  # How to format each example
    suffix="text: {input}",      # The actual query (comes after examples)
    input_variables=["input"]    # Variables we need to provide
)

print("✅ FewShotPromptTemplate created successfully!")

In [ ]:
# ============================================================================
# VIEWING THE FINAL PROMPT
# ============================================================================

final_prompt = prompt.format(input="The MunichDeals experience was just awesome!")

print("📝 Complete Few-Shot Prompt:")
print("=" * 60)
print(final_prompt)
print("=" * 60)

---

## 🧠 Chain-of-Thought (CoT) Prompting

**Chain-of-Thought prompting** goes beyond few-shot by showing the model the **reasoning process**, not just the final answer.

| Technique | Shows | Example |
|-----------|-------|---------|
| Few-shot | Input → Output | "Great food!" → positive |
| Chain-of-Thought | Input → Reasoning → Output | "Great food!" → "expresses satisfaction" → positive |

CoT helps with:
- Complex reasoning tasks
- Reducing errors
- Making outputs more explainable

In [ ]:
# ============================================================================
# CHAIN-OF-THOUGHT TEMPLATE
# ============================================================================
# This template includes the REASONING behind each classification
# The Q&A format guides the model through the thought process
# ============================================================================

TEMPLATE_COT = """
Interprete the text and evaluate the text. Determine if the text has a positive, neutral, or negative sentiment. Also, identify the subject of the text in one word.

Format the output as JSON with the following keys:
sentiment
subject

text: {input}

Chain-of-Thought Prompts:
Let's start by evaluating a statement. Consider: "The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable." How does this make you feel about BellaVista?
 It sounds like a positive review for BellaVista.

Based on the positive nature of that statement, how would you format your response?
 {{ "sentiment": "positive", "subject": "BellaVista" }}

Now, think about this: "SeoulSavor was okay. The bibimbap was good but the bulgogi was a bit too sweet for my taste." Does this give a strong feeling either way?
 Not particularly. It seems like a mix of good and not-so-good elements, so it's neutral.

Given the neutral sentiment, how should this be presented?
 {{ "sentiment": "neutral", "subject": "SeoulSavor" }}

Lastly, ponder on this: "I was let down by MunichMeals. The potato salad lacked flavor and the staff seemed uninterested." What's the overall impression here?
 The statement is expressing disappointment and dissatisfaction.

And if you were to categorize this impression, what would it be?
 {{ "sentiment": "negative", "subject": "MunichMeals" }}
"""

print("✅ Chain-of-Thought template created!")
print("💡 Notice how each example shows the REASONING, not just the answer.")

---

## 🔧 Prompt Composition with PipelinePromptTemplate

For complex prompts, you can **compose smaller templates** into a larger one. This enables:
- **Reusability**: Use the same introduction or examples across multiple prompts
- **Maintainability**: Update one component without touching others
- **Flexibility**: Mix and match components for different use cases

In [ ]:
# ============================================================================
# PIPELINE PROMPT TEMPLATE - Composable Prompts
# ============================================================================
# Break a complex prompt into reusable components:
# 1. Introduction - Task description
# 2. Example - CoT demonstration
# 3. Execution - The actual input to process
# ============================================================================

from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate,FewShotPromptTemplate

# -----------------------------------------------------------------------------
# Component 1: Introduction (Task Description)
# -----------------------------------------------------------------------------
introduction_template = """
Interprete the text and evaluate the text. Determine if the text has a positive, neutral, or negative sentiment. Also, identify the subject of the text in one word.
"""
introduction_prompt = PromptTemplate.from_template(introduction_template)

# -----------------------------------------------------------------------------
# Component 2: Example (Chain-of-Thought Demonstration)
# -----------------------------------------------------------------------------
example_template = """
Chain-of-Thought Prompts:
Let's start by evaluating a statement. Consider: "{example_text}". How does this make you feel about {example_subject}?
Response: {example_evaluation}

Based on the {example_sentiment} nature of that statement, how would you format your response?
Response: {example_format}
"""
example_prompt = PromptTemplate.from_template(example_template)

# -----------------------------------------------------------------------------
# Component 3: Execution (The Actual Query)
# -----------------------------------------------------------------------------
execution_template = """
Now, execute this process for the text: "{input}".
"""
execution_prompt = PromptTemplate.from_template(execution_template)

# -----------------------------------------------------------------------------
# Combine Components into Final Prompt
# -----------------------------------------------------------------------------
full_template = """{introduction}

{example}

{execution}"""
full_prompt = PromptTemplate.from_template(full_template)

# Create the pipeline
input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("execution", execution_prompt)
]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, 
    pipeline_prompts=input_prompts
)

print("✅ PipelinePromptTemplate created with 3 components!")
print(f"📋 Required input variables: {pipeline_prompt.input_variables}")




In [ ]:
# ============================================================================
# USING THE PIPELINE PROMPT
# ============================================================================

formatted_pipeline = pipeline_prompt.format(
    # Example component variables
    example_text="The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable.",
    example_subject="BellaVista",
    example_evaluation="It sounds like a positive review for BellaVista.",
    example_sentiment="positive",
    example_format='{ "sentiment": "positive", "subject": "BellaVista" }',
    # Execution component variable
    input="The new restaurant downtown has bland dishes and the wait time is too long."
)

print("📝 Composed Prompt from Pipeline:")
print("=" * 60)
print(formatted_pipeline)
print("=" * 60)

---

## 💾 Serializing Prompts (Save & Load)

You can **save prompts to files** (YAML or JSON) and load them later. This is useful for:
- Version control of prompts
- Sharing prompts across projects
- A/B testing different prompts

> **Note:** `PipelinePromptTemplate` serialization is not yet fully supported.

In [ ]:
# ============================================================================
# SAVING PROMPTS TO FILES
# ============================================================================
# LangChain supports YAML and JSON formats
# ============================================================================

prompt = PromptTemplate(input_variables=["input"], template="Tell me a joke about {input}")

# Save to YAML (human-readable)
prompt.save("prompt.yaml")
print("✅ Saved to prompt.yaml")

# Save to JSON (easy to parse programmatically)
prompt.save("prompt.json")
print("✅ Saved to prompt.json")

In [ ]:
# ============================================================================
# LOADING PROMPTS FROM FILES
# ============================================================================

from langchain.prompts import load_prompt

# Load from YAML
prompt_yaml = load_prompt("prompt.yaml")
print("📄 Loaded from YAML:")
print(prompt_yaml.format(input="chickens"))

In [ ]:
# Load from JSON
prompt_json = load_prompt("prompt.json")
print("\n📄 Loaded from JSON:")
print(prompt_json.format(input="cows"))

# ============================================================================
# 📝 KEY TAKEAWAYS FROM THIS NOTEBOOK:
# ============================================================================
# 1. PromptTemplate: Create dynamic prompts with {variables}
# 2. Few-Shot Prompting: Include examples to guide LLM behavior
# 3. FewShotPromptTemplate: Modular example management
# 4. Chain-of-Thought: Show reasoning, not just answers
# 5. PipelinePromptTemplate: Compose complex prompts from parts
# 6. Serialization: Save/load prompts with .save() and load_prompt()
# ============================================================================